XYbar:  [ 0.01931274  0.05841159]
check:  [ -1.76066806e-09   1.07280584e-09]
FF:  [[   0.         -137.73170552]
 [   0.            0.        ]]
GG:  [[ 251.24888897  128.79694706]
 [ -18.77371404  305.15657848]]
HH:  [[ -47.53544329 -143.77149749]
 [-115.38666166  -37.37975925]]
LL:  [[ 468.11869861]
 [ 200.17332552]]
MM:  [[-546.59124782]
 [-405.73709328]]
Sylvester equation solver condition is not satisfied; proceed with the original method...
PP: [[ 0.03530577  0.63635012]
 [ 0.38029487  0.16164297]]
QQ [[ 0.58379203]
 [ 0.73578832]]


In [5]:
# Define Variables
import numpy as np
import scipy.optimize as opt
import math as m

periods = 3
years = 20

dfacty = .96
dfact = dfacty ** 20
beta = dfact

delty = .05
delta = 1 - (1 - delty) ** 20

sigma = 3
alpha = .35
rho = 0.9**20
lone = 1
ltwo = 1
lthree = .2
labor = lone + ltwo + lthree

def Modeldyn2(theta1, params):

    '''
    no p = time 1
    p = time 2
    pp = time 3
    '''
    # unpack theta1
    k2pp, k3pp, k2p, k3p, k2, k3, Zp, Z = theta1
    alpha, beta, sigma, lone, ltwo, lthree, labor = params
    
    # find definitions for now and next period
    K = k2+k3
    Kp = k2p+k3p
    
    
    w = (1-alpha)*np.exp(Z)*((K/labor)**alpha)
    wp = (1-alpha)*np.exp(Zp)*((Kp/labor)**alpha)

    r = alpha*np.exp(Z)*((labor/K)**(1-alpha))-delta
    rp = alpha*np.exp(Zp)*((labor/Kp)**(1-alpha))-delta
    
    c1 = w*lone - k2p
    c2 = w*ltwo + (1+r)*k2-k3p
    c2p = wp*ltwo + (1+rp)*k2p-k3pp
    c3p = wp*lthree + (1+rp)*k3p
    
    # Evaluate Euler equations
    E1 = c1**(-sigma)-beta*(1+rp)*(c2p**(-sigma))
    E2 = c2**(-sigma)-beta*(1+rp)*(c3p**(-sigma))
    
    return np.array([E1, E2])

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.optimize as opt
from LinApp_FindSS import LinApp_FindSS
from LinApp_Deriv import LinApp_Deriv
from LinApp_Solve import LinApp_Solve

# make parameter list to pass to functions
params = alpha, beta, sigma, lone, ltwo, lthree, labor

# set LinApp parameters
Zbar = np.array([0.])
nx = 2
ny = 0
nz = 1
Sylv = 1

# take a guess for steady state values of k and ell
guessXY = np.array([.025, .025])

# find the steady state values using LinApp_FindSS
XYbar = LinApp_FindSS(Modeldyn2, params, guessXY, Zbar, nx, ny)
(k2bar, k3bar) = XYbar
print ('XYbar: ', XYbar)

# set up steady state input vector
theta1 = np.array([k2bar, k3bar, k2bar, k3bar, k2bar, k3bar, 0, 0])

# check SS solution
check = Modeldyn2(theta1, params)
print ('check: ', check)
if np.max(np.abs(check)) > 1.E-6:
    print ('Have NOT found steady state')

# find the derivatives matrices
[AA, BB, CC, DD, FF, GG, HH, JJ, KK, LL, MM, WW, TT] = \
    LinApp_Deriv(Modeldyn2, params, theta1, nx, ny, nz, logX=0)
print('FF: ', FF)
print('GG: ', GG)
print('HH: ', HH)
print('LL: ', LL)
print('MM: ', MM)

# set value for NN    
NN = dfacty
    
# find the policy and jump function coefficients
PP, QQ, UU, RR, SS, VV = \
    LinApp_Solve(AA,BB,CC,DD,FF,GG,HH,JJ,KK,LL,MM,WW,TT,NN,Zbar,Sylv)
print ('PP:',PP)
print ('QQ', QQ)

k2 = np.zeros((10000,60))
k3 = np.zeros((10000,60))
k2[:,0] = 0.8*k2bar
k3[:,0] = 1.1*k3bar

mu, sigma = 0, 0.02
eps = np.random.normal(mu, sigma, size=(10000, 61))

z = np.zeros((10000,61))
z[:,0] = np.zeros(10000)
for i in range(1, 61):
    z[:,i] = rho*z[:,i-1] + eps[:,i]

Xn =
zj =
Xbar = 
for i in range(1, 60):
    Xn = Xbar + np.dot(PP, (Xn-Xbar)) + np.dot(QQ, zj)
    
    #k2[:, i] = PP[0, 0] * k2[:, i - 1] + QQ[0, 0] * z[:, i] 
    #k3[:, i] = PP[1, 0] * k3[:, i - 1] + QQ[1, 0] * z[:, i] 


K_sum = k2 + k3


labels = np.array(['k2', 'k3', 'K'])
values = np.array([k2, k3, K_sum])

for i in range(len(values)):
    time = np.linspace(1,60,60)
    PCT5 = np.percentile(values[i], 5, axis=0)
    PCT95 = np.percentile(values[i], 95, axis=0)
    average = np.mean(values[i], axis=0)
    plt.subplot(str(22) + str(i+1))
    plt.plot(time, PCT5, 'b--', label='5th Percentile')
    plt.plot(time, PCT95, 'b--', label='95th Percentile')
    plt.plot(time, average, 'k-', label='Average')
    plt.xlabel('Period')
    plt.ylabel(labels[i])
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()




SyntaxError: invalid syntax (<ipython-input-5-9b6413c3d0db>, line 121)